# <font color='264CC7'> notion_uploader </font>

En construcción.

Los paquetes necesarios son:

In [1]:
# !pip install pymupdf 
# !pip install openai
# !pip install notion-client

In [2]:
# Paquetes necesarios
import fitz  # PyMuPDF
from openai import OpenAI 
from notion_client import Client

In [3]:
# Asignar clave API de OpenAI
from keys import OPENAI_API_KEY, NOTION_TOKEN, NOTION_DATABASE_ID

client = OpenAI(api_key=OPENAI_API_KEY)
notion = Client(auth=NOTION_TOKEN)

---
## <font color='264CC7'> Lectura del PDF y extracción de datos </font>


In [4]:
# 📄 Función: Extraer texto desde PDF
def extraer_texto_desde_pdf(ruta_pdf):
    texto = ""
    with fitz.open(ruta_pdf) as doc:
        for pagina in doc:
            texto += pagina.get_text()
    return texto

# 🤖 Función: Analizar con GPT
def analizar_con_gpt(texto):
    prompt = """
Extrae la siguiente información del texto de un artículo académico:

- Título
- Autor principal
- Revista
- Año
- Tema (un tema como: Educación, Matemática, Ciencia de Datos, etc.)
- Subtema (un subtema, máximo 2 palabras)
- Dos ideas principales
- Tres citas textuales

Devuelve un JSON con la estructura:
{
  "titulo": "...",
  "autor_principal": "...",
  "revista": "...",
  "año": ...,
  "tema": ["..."],
  "subtema": ["..."],
  "ideas_principales": ["..."],
  "citas_textuales": ["..."]
}

Texto:
""" + texto
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={ "type": "json_object" },
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return eval(response.choices[0].message.content)

In [5]:
ruta = "datos/test.pdf"
texto_extraido = extraer_texto_desde_pdf(ruta)
# Cantidad de caracteres extraídos
print(len(texto_extraido))

32994


In [6]:
json_data = analizar_con_gpt(texto_extraido)

print(json_data)

{'titulo': 'Minería de datos en la gestión académica de la educación superior', 'autor_principal': 'Harold Elbert Escobar Terán', 'revista': 'Didasc@lia: Didáctica y Educación', 'año': 2017, 'tema': ['Educación'], 'subtema': ['Minería de datos'], 'ideas_principales': ['La minería de datos puede mejorar la eficiencia académica y ayudar a personalizar el aprendizaje de los estudiantes.', 'La implementación de técnicas de minería de datos en la educación permite identificar factores que influyen en el rendimiento y la deserción escolar.'], 'citas_textuales': ['La preocupación de los administradores educativos por mejorar la eficiencia académica y cumplir con el encargo social de potenciar el talento humano.', 'La minería de datos educativos permite el desarrollo de métodos para el descubrimiento de tipos particulares de datos que provienen de plataformas educativas.', 'La implementación de minería de datos educativos se está constituyendo en una de las herramientas más prácticas hoy en dí

---
## <font color='264CC7'> Envío a Notion </font>

In [7]:
# json_data = """
# {
#     "titulo": "Minería de datos en la gestión académica de la educación superior",
#     "autor_principal": "Harold Elbert Escobar Terán",
#     "revista": "Didasc@lia: Didáctica y Educación",
#     "año": "2017",
#     "tema": "Educación",
#     "subtema": "Minería de datos",
#     "ideas_principales": [
#         "La minería de datos permite mejorar.",
#         "La implementación de técnicas de minería."
#     ],
#     "citas_textuales": [
#         "La minería de datos educativos.",
#         "El análisis de estos factores.",
#         "Estas técnicas aplicadas en los sistemas educativos tributa."
#     ]
# }"""

In [8]:
# 📤 Función: Enviar a Notion
def enviar_a_notion(datos):
    propiedades = {
        "Título": {"title": [{"text": {"content": datos["titulo"]}}]},
        "Autor principal": {"rich_text": [{"text": {"content": datos["autor_principal"]}}]},
        "Revista": {"rich_text": [{"text": {"content": datos["revista"]}}]},
        "Año": {"number": int(datos["año"])},
        "Lectura": {"select": {"name": "Completa"}},
        "Tema": {"multi_select": [{"name": t.strip()} for t in datos["tema"]]},
        "Subtema": {"multi_select": [{"name": s.strip()} for s in datos["subtema"]]}
    }

    children = []

    if datos["ideas_principales"]:
        children.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "Ideas principales"}}]}
        })
        for idea in datos["ideas_principales"]:
            children.append({
                "object": "block",
                "type": "bulleted_list_item",
                "bulleted_list_item": {"rich_text": [{"text": {"content": idea}}]}
            })

    if datos["citas_textuales"]:
        children.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "Citas"}}]}
        })
        for cita in datos["citas_textuales"]:
            children.append({
                "object": "block",
                "type": "quote",
                "quote": {"rich_text": [{"text": {"content": cita}}]}
            })

    notion.pages.create(
        parent={"database_id": NOTION_DATABASE_ID},
        properties=propiedades,
        children=children
    )

In [9]:
enviar_a_notion(json_data)